In [6]:
#Fetches the data for the current Série A


import pandas as pd
import requests
from bs4 import BeautifulSoup
import csv

# Make a request to the webpage URL and extract data from fourth table
url = 'https://www.chancedegol.com.br/brb23.htm'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
table = soup.find_all('table')[7]

# Loop through the rows of the table and extract the data from the cells
data = []
for row in table.find_all('tr'):
    cols = row.find_all('td')
    cols = [col.text.strip() for col in cols]
    data.append(cols)

# Write the data to a CSV file
with open('dataBRB23raw.csv', 'w', newline='', encoding='utf-8-sig') as file:
    writer = csv.writer(file)
    writer.writerows(data)
 

In [8]:
df = pd.read_csv('dataBRB23raw.csv')
df

,Data,Mandante,Unnamed: 2,Visitante,Vitória doMandante,Empate,Vitória doVisitante
0,14/04/2023,Guarani,4x1,Avaí,49.0 %,26.4 %,24.7 %
1,14/04/2023,Ituano,2x0,Ceará,31.8 %,27.3 %,40.9 %
2,15/04/2023,Vila Nova GO,2x1,Grêmio Novorizontino,38.1 %,31.0 %,31.0 %
3,15/04/2023,Mirassol,1x0,Chapecoense,54.0 %,26.6 %,19.4 %
4,15/04/2023,Sampaio Corrêa,3x3,Atlético GO,37.4 %,24.7 %,37.9 %
...,...,...,...,...,...,...,...
183,23/07/2023,Guarani,1x0,Atlético GO,54.4 %,23.4 %,22.3 %
184,23/07/2023,Sampaio Corrêa,1x2,Sport,21.9 %,25.2 %,52.9 %
185,23/07/2023,Juventude,1x0,Ceará,37.0 %,26.8 %,36.2 %
186,24/07/2023,Vitória,1x0,Chapecoense,53.9 %,24.1 %,22.0 %


In [9]:
# Cleans and organizes the Data.


df.rename(columns={'Mandante':'Casa','Visitante':'Fora'}, inplace =True)
#changes dtype to datetime in first dataframe
df['Data'] = pd.to_datetime(df['Data'], infer_datetime_format=True, errors='coerce')

# Creates a new column with the for the competition name
df = df.assign(Comp='Série B')

# Separates the results of second dataframe into new columns
new = df["Unnamed: 2"].str.split("x", n = 1, expand = True)
df["GC"]= new[0] 
df["GF"]= new[1]

# Comparing scores to display the results in the second dataframe
df.loc[df['GC'] == df['GF'], 'Res'] = 'E'
df.loc[df['GC'] > df['GF'], 'Res'] = 'VC' 
df.loc[df['GC'] < df['GF'], 'Res'] = 'VV'

# Create a new column to store the season information in the second data frame
df['Data'] = pd.to_datetime(df['Data'], infer_datetime_format=True, errors='coerce')
df['Temp'] = df['Data'].dt.year

# Organizes de columns of the sacond dataframe
new_order = ['Comp','Temp','Data','Casa','GC', 'GF','Fora','Res']
df= df.reindex(columns=new_order)


# Padronizes the names of the teams
df['Casa'] = df['Casa'].str.strip()
df['Fora'] = df['Fora'].str.strip()

df.replace({'Atlético GO': 'Atlético-GO', 
             'Botafogo SP': 'Botafogo-SP',
             'Grêmio Novorizontino': 'Novorizontino', 
             'Vila Nova GO': 'Vila Nova-GO',
            }, inplace=True)
df.dropna(axis=0, how='all', inplace=True)

# Sorts values by date
df = df.sort_values(by='Data')

# Atributes a pontuation to each game
df['PC'] = df.apply(lambda x: 3 if x['Res'] == 'VC' else
                              1 if x['Res'] == 'E' else 0, axis=1)
df['PF'] = df.apply(lambda x: 3 if x['Res'] == 'VV' else
                              1 if x['Res'] == 'E' else 0, axis=1)

# Atributes a number of games for each season
df['J'] = df.groupby('Temp').cumcount() + 1

# Corrects any possible issue before saving as new file
df = df.dropna()



C:\Users\Cristina Jorge\AppData\Local\Temp\ipykernel_9624\179149278.py:6: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['Data'] = pd.to_datetime(df['Data'], infer_datetime_format=True, errors='coerce')
C:\Users\Cristina Jorge\AppData\Local\Temp\ipykernel_9624\179149278.py:6: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Data'] = pd.to_datetime(df['Data'], infer_datetime_format=True, errors='coerce')
C:\Users\Cristina Jorge\AppData\Local\Temp\ipykernel_9624\179149278.py:22: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/p

In [10]:
df.tail(10)

,Comp,Temp,Data,Casa,GC,GF,Fora,Res,PC,PF,J
178,Série B,2023,2023-07-21,CRB,1,0,Ponte Preta,VC,3,0,179
179,Série B,2023,2023-07-22,Vila Nova-GO,1,1,ABC,E,1,1,180
180,Série B,2023,2023-07-22,Criciúma,0,1,Novorizontino,VV,0,3,181
181,Série B,2023,2023-07-22,Ituano,0,1,Avaí,VV,0,3,182
182,Série B,2023,2023-07-22,Mirassol,2,0,Tombense,VC,3,0,183
184,Série B,2023,2023-07-23,Sampaio Corrêa,1,2,Sport,VV,0,3,184
183,Série B,2023,2023-07-23,Guarani,1,0,Atlético-GO,VC,3,0,185
185,Série B,2023,2023-07-23,Juventude,1,0,Ceará,VC,3,0,186
186,Série B,2023,2023-07-24,Vitória,1,0,Chapecoense,VC,3,0,187
187,Série B,2023,2023-07-24,Londrina,1,2,Botafogo-SP,VV,0,3,188


In [11]:
#Saves de df in a new file
df.to_csv('dataBRB23.csv', index=False)